In [16]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [17]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings
import itertools

# Libs de Modelagem
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.datasets import get_rdataset
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS

# Libs de Avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm # Para uma barra de progresso no nosso loop
from IPython.display import display

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [18]:
# =========================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES (SETUP E PROCESSAMENTO)
# =========================================================

In [19]:
def definir_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [20]:
def salvar_dataset(serie, dataset_name):
    dir_path = "./datasets"
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, f"{dataset_name.lower()}.csv")
    df = pd.DataFrame({"date": serie.index, "value": serie.values})
    df.to_csv(file_path, index=False)
    print(f"-> Cópia do dataset '{dataset_name}' salva em: {file_path}")

def carregar_serie(nome):
    print(f"Buscando dados de '{nome}' via statsmodels...")
    nome_base = nome.lower()

    if nome_base == "airpassengers":
        df = get_rdataset("AirPassengers", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1949-01-01", periods=len(df), freq="MS"),
                          name="AirPassengers")
    elif nome_base == "lynx":
        df = get_rdataset("lynx", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1821", periods=len(df), freq="A"), name="Lynx")
    elif nome_base == "co2":
        df = get_rdataset("CO2", package="datasets").data
        df = df.ffill()
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1958-03-29", periods=len(df), freq="MS"),
                          name="CO2")
    elif nome_base == "sunspots":
        df = get_rdataset("sunspots", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1749-01-01", periods=len(df), freq="MS"),
                          name="Sunspots")
    elif nome_base == "austres":
        df = get_rdataset("austres", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1971-03-01", periods=len(df), freq="QS-MAR"),
                          name="AustralianResidents")
    elif nome_base == "nottem":
        df = get_rdataset("nottem", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1920-01-01", periods=len(df), freq="MS"),
                          name="Nottingham")
    else:
        raise ValueError(f"Série '{nome}' não reconhecida.")

    salvar_dataset(serie, nome)
    return serie

In [21]:
def dividir_serie_temporal(serie, percentual_treino=0.7, percentual_validacao=0.15):
    tamanho_total = len(serie)
    if tamanho_total < 20: 
        percentual_treino=0.8
        percentual_validacao=0.0
    ponto_corte_treino = int(tamanho_total * percentual_treino)
    ponto_corte_validacao = int(tamanho_total * (percentual_treino + percentual_validacao))
    treino = serie.iloc[:ponto_corte_treino]
    validacao = serie.iloc[ponto_corte_treino:ponto_corte_validacao]
    teste = serie.iloc[ponto_corte_validacao:]
    return treino, validacao, teste

def preparar_dados_para_neuralforecast(serie, nome_serie):
    df = serie.reset_index()
    df.columns = ['ds', 'y']
    df['unique_id'] = nome_serie
    return df

In [22]:
# =========================================================
# SEÇÃO 3: FUNÇÕES PARA CÁLCULO DE MÉTRICAS E MODELAGEM
# =========================================================

In [23]:
def calculate_mape(y_true, y_pred):
    """Calcula o Mean Absolute Percentage Error (MAPE)."""
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def calculate_mase(y_true, y_pred, y_train):
    """Calcula o Mean Absolute Scaled Error (MASE)."""
    n = len(y_train)
    if n <= 1: return np.nan
    d = np.sum(np.abs(y_train[1:] - y_train[:-1])) / (n - 1)
    if d == 0: return np.inf
    errors = np.mean(np.abs(y_true - y_pred))
    return errors / d

def calcular_metricas(y_true, y_pred, y_train):
    """Calcula um dicionário com todas as métricas de erro."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = calculate_mape(y_true, y_pred)
    mase = calculate_mase(y_true, y_pred, y_train)
    return {'RMSE': rmse, 'MAPE': mape, 'MASE': mase}

In [24]:
def treinar_e_prever_hys_mf(treino, validacao, teste, ordem_arima, seed, freq, input_size, h, max_steps=100):
    """Implementa a metodologia HyS-MF com a abordagem MIMO (mais estável) para os resíduos."""
    # 1. Treina o ARIMA e faz a previsão recursiva
    treino_validacao_arima = pd.concat([treino, validacao]).asfreq(freq)
    modelo_arima = ARIMA(treino_validacao_arima, order=ordem_arima).fit()
    preds_arima = modelo_arima.predict(start=teste.index[0], end=teste.index[-1])
    
    # 2. Obtém os resíduos do ARIMA
    residuos = modelo_arima.resid
    df_residuos_nf = preparar_dados_para_neuralforecast(residuos, "residuos")

    # 3. Treina UM modelo N-BEATS para prever TODOS os 'h' passos dos resíduos de uma vez
    modelos_residuos = [NBEATS(input_size=input_size, h=h, max_steps=max_steps, scaler_type='standard', random_seed=seed)]
    nf_residuos = NeuralForecast(models=modelos_residuos, freq=freq)
    nf_residuos.fit(df=df_residuos_nf)
    
    preds_residuos = nf_residuos.predict()['NBEATS'].values

    # 4. Previsão Híbrida = Previsão ARIMA + Previsão dos Resíduos
    preds_hibrido = preds_arima.values + preds_residuos
    
    return preds_arima, preds_hibrido


In [25]:
# =========================================================
# SEÇÃO 4: FUNÇÃO PRINCIPAL DO EXPERIMENTO (ENCAPSULADA)
# =========================================================

In [26]:
def executar_experimento_completo(nome_da_serie):
    """Executa o pipeline completo de modelagem e avaliação para uma única série temporal."""
    try:
        # --- Configurações do Experimento ---
        N_CICLOS = 5
        ORDEM_ARIMA = (3, 1, 2)
        SEED_INICIAL = 42
        MAX_INPUT_SIZE = 50 

        # --- Preparação dos Dados ---
        serie_completa = carregar_serie(nome_da_serie)
        treino, validacao, teste = dividir_serie_temporal(serie_completa)
        
        if len(teste) < 2:
            print(f"AVISO: Conjunto de teste para '{nome_da_serie}' é muito pequeno. Pulando.")
            return None

        freq = serie_completa.index.freqstr or pd.infer_freq(serie_completa.index)
        if freq is None:
            print(f"ERRO: Não foi possível determinar a frequência para '{nome_da_serie}'. Pulando.")
            return None
        
        df_treino_validacao_nf = pd.concat([preparar_dados_para_neuralforecast(treino, nome_da_serie), 
                                            preparar_dados_para_neuralforecast(validacao, nome_da_serie)])
        
        h = len(teste)
        input_size = min(2 * h, MAX_INPUT_SIZE)
        resultados_finais = []

        # --- Loop Principal de Execução ---
        for i in range(N_CICLOS):
            definir_seed(SEED_INICIAL + i)
            try:
                # Chama a função que executa os modelos ARIMA e Híbrido
                preds_arima, preds_hibrido = treinar_e_prever_hys_mf(treino, validacao, teste, ORDEM_ARIMA, SEED_INICIAL + i, freq, input_size, h, max_steps=100)
                
                metricas_hibrido = calcular_metricas(teste.values, preds_hibrido, treino.values)
                metricas_hibrido['modelo'] = 'Híbrido (HyS-MF)'
                resultados_finais.append(metricas_hibrido)
                
                metricas_arima = calcular_metricas(teste.values, preds_arima.values, treino.values)
                metricas_arima['modelo'] = 'ARIMA'
                resultados_finais.append(metricas_arima)

                # Executa o modelo N-BEATS puro
                modelos_nbeats = [NBEATS(input_size=input_size, h=h, max_steps=100, scaler_type='standard', random_seed=SEED_INICIAL + i)]
                nf_nbeats = NeuralForecast(models=modelos_nbeats, freq=freq)
                nf_nbeats.fit(df=df_treino_validacao_nf)
                preds_nbeats = nf_nbeats.predict()['NBEATS'].values
                metricas_nbeats = calcular_metricas(teste.values, preds_nbeats, treino.values)
                metricas_nbeats['modelo'] = 'N-BEATS'
                resultados_finais.append(metricas_nbeats)
            except Exception as e:
                print(f"AVISO: Ciclo {i+1} para '{nome_da_serie}' falhou. Erro: {e}")
                continue

        if not resultados_finais: return None
        
        df_resultados = pd.DataFrame(resultados_finais)
        df_sumario = df_resultados.groupby('modelo').agg(['mean', 'std'])
        return df_sumario.reindex(columns=['RMSE', 'MAPE', 'MASE'], level=0)
        
    except Exception as e:
        print(f"ERRO GERAL no processamento do dataset '{nome_da_serie}': {e}")
        return None

In [27]:
# =========================================================
# SEÇÃO 5: ORQUESTRADOR DOS EXPERIMENTOS
# =========================================================

In [ ]:
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'austres', 'lynx', 'sunspots']
resultados_gerais = {}

for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando todos os datasets"):
    sumario_dataset = executar_experimento_completo(dataset)
    if sumario_dataset is not None:
        resultados_gerais[dataset] = sumario_dataset

Processando todos os datasets:   0%|          | 0/6 [00:00<?, ?it/s]

Buscando dados de 'AirPassengers' via statsmodels...
-> Cópia do dataset 'AirPassengers' salva em: ./datasets\airpassengers.csv


Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
3.0 K     Non-trainable params
2.5 M     Total params
10.064    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s, v_num=350, train_loss_step=0.0255, train_loss_epoch=0.0255]

`Trainer.fit` stopped: `max_steps=100` reached.


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  3.50it/s, v_num=408, train_loss_step=0.106, train_loss_epoch=0.106]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 90.89it/s] 

Processando todos os datasets:  50%|█████     | 3/6 [13:25<13:27, 269.33s/it]


Buscando dados de 'austres' via statsmodels...


In [ ]:
# =========================================================
# SEÇÃO 6: GERAÇÃO DE RELATÓRIOS FINAIS
# =========================================================

In [ ]:
print("\n\n" + "="*60)
print("     RELATÓRIO FINAL: COMPARAÇÃO DE MODELOS ENTRE DATASETS")
print("="*60)

if not resultados_gerais:
    print("Nenhum experimento foi concluído com sucesso para gerar um relatório.")
else:
    lista_dfs_finais = []
    for dataset, df_sumario in resultados_gerais.items():
        df_mean = df_sumario.xs('mean', axis=1, level=1).copy()
        df_mean['dataset'] = dataset
        lista_dfs_finais.append(df_mean.reset_index())

    df_final_comparativo = pd.concat(lista_dfs_finais).rename(columns={'modelo': 'Modelo'})
    
    # Criando a tabela pivotada para o relatório
    df_pivot_rmse = df_final_comparativo.pivot(index='dataset', columns='Modelo', values='RMSE')
    
    def destacar_menor_na_linha(s):
        is_min = s == s.min()
        return ['background-color: lightgreen' if v else '' for v in is_min]

    print("\nComparativo de Desempenho (RMSE Médio):")
    styled_final = (df_pivot_rmse.style
                    .format('{:.2f}')
                    .apply(destacar_menor_na_linha, axis=1)
                    .set_caption("O verde indica o melhor modelo (menor RMSE) para cada dataset."))
    display(styled_final)



     RELATÓRIO FINAL: COMPARAÇÃO DE MODELOS ENTRE DATASETS
Nenhum experimento foi concluído com sucesso para gerar um relatório.
